In [ ]:
from envutils import ENV, load_env, get_argparser, log

from pathlib import Path
import os
import tifffile
import matplotlib.pyplot as plt
import numpy as np

from careamics import CAREamist
import careamics.dataset.tiling as tiling
from careamics.prediction_utils import stitch_prediction_single
from typing import List


/localscratch/conda_environments/ai4life_calcium_imaging_denoising/lib/python3.10/site-packages/skimage/metrics/simple_metrics.py:2: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.23.0)
  from scipy.stats import entropy


In [2]:

def tiled_prediction(image: np.ndarray, model, patch_shape: List[int]=(16, 64, 64), patch_batch_size:int = 8, axes="ZYX"):
    """
        N2V batched patch-wise prediction.
    """

    tiles_generator = tiling.extract_tiles(arr=image[None, None, ...], tile_size=patch_shape, overlaps=list(p//2 + 1 for p in patch_shape))
    
    pred_patches = list()
    patch_info = list()
    current_batch = list()

    for current_tile, current_info in tiles_generator:
        patch_info.append(current_info)
        current_batch.append(current_tile)

        # Processing batch (if full)
        if len(current_batch) == patch_batch_size:
            current_batch = np.concatenate(current_batch)
            pred_patches += model.predict(current_batch, data_type='array', axes=f'S{axes}')
            current_batch = []  # Create new empty batch 
            log.debug(f"Predicted {len(pred_patches)} tiles...")
    # Process last batch if is n_tiles % batch_size > 0
    if len(current_batch) > 0:
        pred_patches += model.predict(np.concatenate(current_batch), data_type='array', axes=f'S{axes}')
        current_batch = []  # Create new empty batch
        log.debug(f"Predicted {len(pred_patches)} tile. Done.")

    pred_patches = list(np.array(pred_patches)[:, None, ...])
    return stitch_prediction_single(pred_patches, patch_info)


dataset_name = "train"
dataset_folder = "/localscratch/calcium_imaging_dataset/calcium_imaging/refined"
models_folder = "./models"
output_folder = "./output"
experiment_name = "TEST_TRAIN_N2V"
patch_size_z=None
patch_size=64
batch_size=16
axes="TYX"
model_ckpt='last.ckpt'


# Variables and Paths
model_folder = os.path.join(models_folder, experiment_name)
ckpt_path = os.path.join(model_folder, 'checkpoints', model_ckpt)
output_path = os.path.join(output_folder, experiment_name)
os.makedirs(output_path, exist_ok=True)

# instantiate a CAREamist
careamist = CAREamist(
    ckpt_path,
    work_dir=model_folder, 
)

# Predicts over files
for tiff_path_in in sorted(Path(dataset_folder).joinpath(dataset_name).glob(f'*.tif*')):

    print(f"Predicting file {tiff_path_in}")
    tiff_in = tifffile.imread(tiff_path_in)

    tiff_out = []
    for slice in tiff_in:
        tiff_out += careamist.predict(slice, data_type='array', axes="YX")

    tiff_out = np.concatenate(tiff_out).squeeze()

    tiff_path_out = Path.joinpath(Path(output_path), tiff_path_in.name)
    print(f"Writing prediction to {tiff_path_out}")
    os.makedirs(output_path, exist_ok=True)
    tifffile.imwrite(tiff_path_out, tiff_out)



/localscratch/conda_environments/ai4life_calcium_imaging_denoising/lib/python3.10/site-packages/careamics/model_io/model_io_utils.py:74: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental f

Predicting file /localscratch/calcium_imaging_dataset/calcium_imaging/refined/train/20ms_20%_Yoda1_005.tif


INFO:pytorch_lightning.utilities.rank_zero:You are using a CUDA device ('NVIDIA A40-16Q') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/localscratch/conda_environments/ai4life_calcium_imaging_denoising/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Predicting DataLoader 0: 100%|██████████| 1/1 [00:01<00:00,  0.88it/s]

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 25.20it/s]


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 22.67it/s]

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 22.88it/s]
Writing prediction to output/TEST_TRAIN_N2V/20ms_20%_Yoda1_006.tif


In [3]:
tiff_path_out

PosixPath('output/TEST_TRAIN_N2V/20ms_20%_Yoda1_008.tif')

In [ ]:
if __name__ == "__main__":

    parser = get_argparser(description="Predict N2V for Calcium Imaging Denoising")

    parser.add_argument('-c', '--ckpt', '--checkpoint', type=str, default='last.ckpt', help="Model .ckpt filename to use for prediction. Defaults to last.ckpt.")
    parser.add_argument('--dataset_name', type=str, help='Dataset Name, as subfolder of the dataset directory containing the .tif files')
    parser.add_argument('--experiment_name', type=str, help='Name of the experiment. Will be used to create corresponding subfolders.')
    parser.add_argument('--patch_size_z', type=int, default=None, help="Patch depth dimension")
    parser.add_argument('--patch_size', type=int, default=64, help="Patch spatial dimension")
    parser.add_argument('--batch_size', type=int, default=16, help="Batch Size")
    parser.add_argument('--axes', type=str, default="YX", help="Axes used to interpret the TIFF files.")

    args = parser.parse_args()
    log.setLevel(args.level)
    load_env(args.env, parser_args=args)

    pred_gen = predict_n2v(dataset_name=args.dataset_name,
                dataset_folder=ENV.get("DATASET_FOLDER"),
                models_folder=ENV.get("MODELS_FOLDER"),
                output_folder=ENV.get("OUTPUT_FOLDER"),
                experiment_name=args.experiment_name,
                patch_size_z=args.patch_size_z,
                patch_size=args.patch_size,
                batch_size=args.batch_size,
                axes=args.axes,
                model_ckpt=args.ckpt
                )
    for _ in pred_gen:
        log.info("Done.")